In [34]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [35]:
e = -100000
X1 = np.linspace(0, 3.0,60)
X2 = np.linspace(0, 1.5, 30)
N1 = X1.shape[0]
N2 = X2.shape[0]
U = np.linspace(-10.0, 10.0, 40)
nU = U.shape[0]

J = 1.625103
m = 0.506
M0 = 0.434
L0 = 0.305
R0 = 0.023
B0 = 16.25163
L = 0.0250103
R = 5.0
Kt = 0.90
Kb = 0.90
g = 9.8
M = J + m*L0*L0/3.0 + M0*L0*L0 + 2*M0*R0*R0/5/Kt
N = m*L0*g/2.0 + M0*L0*g/Kt
B = B0/Kt


class SingleLinkManipulatorEnv:

    def _ns(self, x1, x2, a):
        x3 = (a-Kb*x2)/R
        x1dot = x2
        x2dot = -(N/M)*np.sin(x1) - (B/M)*x2 + (x3/M)
        x1p = x1 + 0.1*x1dot
        x2p = x2 + 0.1*x2dot
        return (x1p, x2p)

    def _quantize(self, x1p, x2p):
        INF = 100000
        d1 = d2 = INF
        x1pd = x2pd = 0
        for i in range(N1):
            if np.abs(x1p - X1[i]) < d1:
                d1 = np.abs(x1p - X1[i])
                x1pd = i
        for j in range(N2):
            if np.abs(x2p - X2[j]) < d2:
                d2 = np.abs(x2p - X2[j])
                x2pd = j
        return (x1pd, x2pd)

    def _reward(self, x1, x2):
        return -np.abs(x1-np.pi/6.0) - np.abs(x2)

    def __init__(self, shape=[N1, N2]):
        self.shape = shape
        nS = np.prod(shape)
        nA = nU

        Y_MAX = shape[0]
        X_MAX = shape[1]

        P = {}
        grid = np.arange(nS).reshape(shape)
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:
            s = it.iterindex
            x, y = it.multi_index
            print("x: " + str(x))
            print("y: " + str(y))
            P[s] = {a: [] for a in range(nA)}

            for a in range(nU):
                x1 = X1[x]
                x2 = X2[y]
                x1p, x2p = self._ns(x1, x2, a)
                # returns index wrt X1 and X2
                x1pd, x2pd = self._quantize(x1p, x2p)
                # calculate x1pd and x2pd as a flat array
                ns = (x1pd)*X_MAX + (x2pd)
                
                P[s][a] = [(1.0, ns, self._reward(X1[x1pd], X2[x2pd]), False)]
            it.iternext()
        self.P = P
        self.nS = nS
        self.nA = nA


In [39]:
def value_iteration(env, gamma, eps):
  """
  Arguments:


  env: custom class having the following parameters
    env.P[s][a] is list of transition tuples (p, s_prime, r, terminal)
    env.nS denotes number of states in the env
    env.nA denotes number of actions in the env
  gamma: discount factor
  eps: error epsilon
  
  Returns:

  A tuple (policy, V)
  policy is optimal
  Shape of policy is [S,A]
  V is value function for optimal policy
  """

  def one_step_lookahead(s, V, log = False):
    A = np.zeros(env.nA)
    for a in range(env.nA):
      for p, s_prime, r, terminal in env.P[s][a]:
        if log:
          print(env.P[s][a])
        A[a] += p*(r + gamma * V[s_prime])
    return A
  
  # init v0
  V = np.zeros(env.nS)
  n_iter = 1
  while n_iter <= 200:
    delV = 0
    # full backup for each state
    for s in range(env.nS):
      # find best action at t+1
      A = one_step_lookahead(s, V)
      best_action_value = np.max(A)

      # calc delta and update value function
      delV = max(delV, np.abs(best_action_value - V[s]))
      V[s] = best_action_value
    # print("Value function at iteration: " + str(n_iter)+ ":")
    # print(V.reshape(env.shape))
    n_iter+=1
    # if delV < eps:
    #   break


  policy = np.zeros([env.nS, env.nA])
  # print("Policy Shape: ")
  # print(policy)
  for s in range(env.nS):
    # find best action for state using t+1
    A = one_step_lookahead(s, V, False)
    best_action = np.argmax(A)
    # print("State: "+str(s)+ " => Action: ", end=" ")
    # print(A)
    # greedily take best action
    policy[s, best_action] = 1.0

  return policy, V

In [40]:
env = SingleLinkManipulatorEnv()
  

x: 0
y: 0
x: 0
y: 1
x: 0
y: 2
x: 0
y: 3
x: 0
y: 4
x: 0
y: 5
x: 0
y: 6
x: 0
y: 7
x: 0
y: 8
x: 0
y: 9
x: 0
y: 10
x: 0
y: 11
x: 0
y: 12
x: 0
y: 13
x: 0
y: 14
x: 0
y: 15
x: 0
y: 16
x: 0
y: 17
x: 0
y: 18
x: 0
y: 19
x: 0
y: 20
x: 0
y: 21
x: 0
y: 22
x: 0
y: 23
x: 0
y: 24
x: 0
y: 25
x: 0
y: 26
x: 0
y: 27
x: 0
y: 28
x: 0
y: 29
x: 1
y: 0
x: 1
y: 1
x: 1
y: 2
x: 1
y: 3
x: 1
y: 4
x: 1
y: 5
x: 1
y: 6
x: 1
y: 7
x: 1
y: 8
x: 1
y: 9
x: 1
y: 10
x: 1
y: 11
x: 1
y: 12
x: 1
y: 13
x: 1
y: 14
x: 1
y: 15
x: 1
y: 16
x: 1
y: 17
x: 1
y: 18
x: 1
y: 19
x: 1
y: 20
x: 1
y: 21
x: 1
y: 22
x: 1
y: 23
x: 1
y: 24
x: 1
y: 25
x: 1
y: 26
x: 1
y: 27
x: 1
y: 28
x: 1
y: 29
x: 2
y: 0
x: 2
y: 1
x: 2
y: 2
x: 2
y: 3
x: 2
y: 4
x: 2
y: 5
x: 2
y: 6
x: 2
y: 7
x: 2
y: 8
x: 2
y: 9
x: 2
y: 10
x: 2
y: 11
x: 2
y: 12
x: 2
y: 13
x: 2
y: 14
x: 2
y: 15
x: 2
y: 16
x: 2
y: 17
x: 2
y: 18
x: 2
y: 19
x: 2
y: 20
x: 2
y: 21
x: 2
y: 22
x: 2
y: 23
x: 2
y: 24
x: 2
y: 25
x: 2
y: 26
x: 2
y: 27
x: 2
y: 28
x: 2
y: 29
x: 3
y: 0
x: 3
y: 1
x: 3
y: 2
x: 3
y: 3


In [41]:
policy, v = value_iteration(env, 0.99, 0.00001)
print("Policy")
print(np.reshape(np.argmax(policy, axis = 1), env.shape))
print("\nValue Function")
print(v.reshape(env.shape))

Policy
[[20 20 21 21 22 22 22 23 23 23 24 24 24 25 25 26 26 26 27 27 27 28 28 29
  29 29 30 30 30 31]
 [21 21 21 22 22 22 23 23 24 24 24 25 25 25 26 26 27 27 27 28 28 28 29 29
  29 30 30 31 31 31]
 [21 22 22 22 23 23 23 24 24 24 25 25 26 26 26 27 27 27 28 28 29 29 29 30
  30 30 31 31 31 32]
 [22 22 22 23 23 24 24 24 25 25 25 26 26 27 27 27 28 28 28 29 29 29 30 30
  31 31 31 32 32 32]
 [22 23 23 23 24 24 24 25 25 26 26 26 27 27 27 28 28 29 29 29 30 30 30 31
  31 31 32 32 33 33]
 [23 23 24 24 24 25 25 25 26 26 27 27 27 28 28 28 29 29 29 30 30 31 31 31
  32 32 32 33 33 34]
 [23 24 24 24 25 25 26 26 26 27 27 27 28 28 29 29 29 30 30 30 31 31 31 32
  32 33 33 33 34 34]
 [24 24 25 25 25 26 26 26 27 27 28 28 28 29 29 29 30 30 31 31 31 32 32 32
  33  0  0  0  0  0]
 [24 25 25 26 26 26 27 27 27 28 28 28 29 29 30  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [25 25 26 26 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0